In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
from scipy.optimize import least_squares
from osgeo import gdal
import sys
# this allows GDAL to throw Python Exceptions
gdal.UseExceptions()

def objective(x):
    #print("inside objective")
    squaredErrorSum = (np.square(r[0] - (a[0,0]*x[0] + a[0,1]*x[1] + a[0,2]*x[2])) + \
                    np.square(r[1] - (a[1,0]*x[0] + a[1,1]*x[1] + a[1,2]*x[2])) + \
                    np.square(r[2] - (a[2,0]*x[0] + a[2,1]*x[1] + a[2,2]*x[2])) + \
                    np.square(r[3] - (a[3,0]*x[0] + a[3,1]*x[1] + a[3,2]*x[2])) + \
                    np.square(r[4] - (a[4,0]*x[0] + a[4,1]*x[1] + a[4,2]*x[2])) + \
                    np.square(r[5] - (a[5,0]*x[0] + a[5,1]*x[1] + a[5,2]*x[2])))
#     print(squaredErrorSum)
    return squaredErrorSum

def constraint1(x):
    return x[0]+x[1]+x[2]-1.0



band1 = gdal.Open("geotiff_band1")
band2 = gdal.Open("geotiff_band2")
band3 = gdal.Open("geotiff_band3")
band4 = gdal.Open("geotiff_band4")
band5 = gdal.Open("geotiff_band5")
band6 = gdal.Open("geotiff_band6")

srcband1 = band1.GetRasterBand(1)
srcband2 = band2.GetRasterBand(1)
srcband3 = band3.GetRasterBand(1)
srcband4 = band4.GetRasterBand(1)
srcband5 = band5.GetRasterBand(1)
srcband6 = band6.GetRasterBand(1)
    
    
srcband1_array = srcband1.ReadAsArray()
srcband2_array = srcband2.ReadAsArray()
srcband3_array = srcband3.ReadAsArray()
srcband4_array = srcband4.ReadAsArray()
srcband5_array = srcband5.ReadAsArray()
srcband6_array = srcband6.ReadAsArray()
print(srcband1_array.shape)
    

# 1 pixel approximation - buildings, water, vegetation
# a = np.matrix([[16.15637,8.264342,8.474085], 
#             [20.16146,6.400146,9.62526],
#             [12.3987,3.213381,4.325268],
#             [37.32146,1.581099,43.28848],
#             [16.64283,0.3415293,15.57259],
#             [15.68166,0.1942849,6.020128]])

#polygonal approximation - Nehal - buildings, water, vegetation
# a = np.matrix([[14.2969, 8.40872, 9.13306], 
#             [12.932, 6.30949, 8.69043],
#             [13.6009, 3.23952, 5.40134],
#             [19.1116, 1.53922, 44.2156],
#             [18.3417, 0.382358, 15.6186],
#             [14.8784, 0.221383, 6.91139]])

#polygonal approximation - Ravi - buildings, vegetation, water
a = np.matrix([[14.393, 9.3877, 8.39868], 
            [13.2142, 8.90877, 6.31175],
            [13.9524, 5.99477, 3.24245],
            [20.8509, 40.0929, 1.15506],
            [19.7301, 15.6481, 0.388785],
            [16.0193, 7.41238, 0.225701]])

# print(a.shape)


## loop for lake region
# for i in range(978,1032):
#     for j in range(742,815):

# r_list = [[srcband1_array[0][0]],
        #           [srcband2_array[0][0]],
        #           [srcband3_array[0][0]],
        #           [srcband4_array[0][0]],
        #           [srcband5_array[0][0]],
        #           [srcband6_array[0][0]]]

        # r_list = [[srcband1_array[850][1090]],
        #           [srcband2_array[850][1090]],
        #           [srcband3_array[850][1090]],
        #           [srcband4_array[850][1090]],
        #           [srcband5_array[850][1090]],
        #           [srcband6_array[850][1090]]]

        #pure water pixel
        # r_list = [[srcband1_array[1106][850]],
        #           [srcband2_array[1106][850]],
        #           [srcband3_array[1106][850]],
        #           [srcband4_array[1106][850]],
        #           [srcband5_array[1106][850]],
        #           [srcband6_array[1106][850]]]


        #pure vegetation
#         r_list = [[srcband1_array[1118][4]],
#                   [srcband2_array[1118][4]],
#                   [srcband3_array[1118][4]],
#                   [srcband4_array[1118][4]],
#                   [srcband5_array[1118][4]],
#                   [srcband6_array[1118][4]]]
# loop for substrate region
for i in range(0,10):
    for j in range(0,10):
        r_list = [[srcband1_array[i][j]],
                  [srcband2_array[i][j]],
                  [srcband3_array[i][j]],
                  [srcband4_array[i][j]],
                  [srcband5_array[i][j]],
                  [srcband6_array[i][j]]]

        #print(r_list)

        r = np.array(r_list)
        #     print(r)
        #     print(r.shape)

        n = 3
        x0 = np.zeros(n)
        x0[0] = 0.33
        x0[1] = 0.33
        x0[2] = 0.33

        print("for pixel: " + str(i) + "," + str(j))
        # show initial objective
        print('Initial Objective: ' + str(objective(x0)))

        # optimize
        b = (0.0,1.0)
        bnds = (b, b, b)

        con1 = {'type': 'eq', 'fun': constraint1} 
        cons = ([con1])

        #SLSQP=Sequential Least SQuares Programming optimization algorithm 

        sol = minimize(objective,x0,method='SLSQP',bounds=bnds,constraints=cons,options={'eps': 1.4901161193847656e-01, 'maxiter' : 50})
        x = sol.x
#         print(x)
        # show final objective
        print('Final Objective: ' + str(objective(x)))

        # print solution
#         print('Solution')
        
        print('x1 = ' + str(x[0]))
        print('x2 = ' + str(x[1]))
        print('x3 = ' + str(x[2]))
        print('sum = ', x[0] + x[1] + x[2])
        print("-----------------------------")
        
        

(1200, 1200)
for pixel: 0,0
Initial Objective: [ 203.70040894]
Final Objective: [ 16.06269836]
x1 = 0.964811672193
x2 = 2.872256662e-10
x3 = 0.0351883275218
sum =  1.0
-----------------------------
for pixel: 0,1
Initial Objective: [ 89.7726059]
Final Objective: [ 5.121984]
x1 = 0.782121002986
x2 = 4.11987398142e-13
x3 = 0.217879002045
sum =  1.00000000503
-----------------------------
for pixel: 0,2
Initial Objective: [ 68.12703705]
Final Objective: [ 1.31441438]
x1 = 0.702943644675
x2 = 8.00590758864e-20
x3 = 0.297056359186
sum =  1.00000000386
-----------------------------
for pixel: 0,3
Initial Objective: [ 62.32710266]
Final Objective: [ 1.72332013]
x1 = 0.66282323868
x2 = 2.80722182486e-09
x3 = 0.337176758513
sum =  1.0
-----------------------------
for pixel: 0,4
Initial Objective: [ 83.92354584]
Final Objective: [ 3.56129384]
x1 = 0.721177930585
x2 = 2.18542209615e-09
x3 = 0.27882206723
sum =  1.0
-----------------------------
for pixel: 0,5
Initial Objective: [ 118.71777344]
F

sum =  1.00000000001
-----------------------------
for pixel: 4,8
Initial Objective: [ 129.95188904]
Final Objective: [ 7.313622]
x1 = 0.835118249348
x2 = 2.01234253937e-09
x3 = 0.164881748788
sum =  1.00000000015
-----------------------------
for pixel: 4,9
Initial Objective: [ 119.80146027]
Final Objective: [ 6.18809319]
x1 = 0.818563413719
x2 = 1.06682001517e-09
x3 = 0.181436598281
sum =  1.00000001307
-----------------------------
for pixel: 5,0
Initial Objective: [ 196.10366821]
Final Objective: [ 5.77914619]
x1 = 0.952343013495
x2 = 3.2869927998e-09
x3 = 0.047656984533
sum =  1.00000000132
-----------------------------
for pixel: 5,1
Initial Objective: [ 575.60638428]
Final Objective: [ 110.76756287]
x1 = 0.999999999996
x2 = 7.00829948831e-12
x3 = 0.0
sum =  1.0
-----------------------------
for pixel: 5,2
Initial Objective: [ 1781.07543945]
Final Objective: [ 865.51275635]
x1 = 0.999999998824
x2 = 7.46517763668e-10
x3 = 4.29195849826e-10
sum =  1.0
-----------------------------


Final Objective: [ 2.13136983]
x1 = 0.632596381557
x2 = 9.47626464455e-20
x3 = 0.367403620301
sum =  1.00000000186
-----------------------------
for pixel: 9,4
Initial Objective: [ 92.9249115]
Final Objective: [ 3.15803075]
x1 = 0.729322818283
x2 = 3.41243351142e-09
x3 = 0.270677178305
sum =  1.0
-----------------------------
for pixel: 9,5
Initial Objective: [ 197.68899536]
Final Objective: [ 9.9017725]
x1 = 0.914429778129
x2 = 6.94663271991e-13
x3 = 0.0855702222903
sum =  1.00000000042
-----------------------------
for pixel: 9,6
Initial Objective: [ 116.2684021]
Final Objective: [ 5.20185757]
x1 = 0.80009421351
x2 = 2.07075413688e-08
x3 = 0.199905767861
sum =  1.00000000208
-----------------------------
for pixel: 9,7
Initial Objective: [ 82.08850098]
Final Objective: [ 2.75808716]
x1 = 0.680411014677
x2 = 2.35540316011e-19
x3 = 0.319588987357
sum =  1.00000000203
-----------------------------
for pixel: 9,8
Initial Objective: [ 86.41248322]
Final Objective: [ 9.74926376]
x1 = 0.586